In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


def show_image(title, img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6, 6))
    plt.imshow(img_rgb)
    plt.title(title)
    plt.axis("off")
    plt.show()


def remove_hair(img, kernel_size=9):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
    _, hair_mask = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    inpainted = cv2.inpaint(img, hair_mask, 1, cv2.INPAINT_TELEA)
    return inpainted, hair_mask


def check_hair_coverage(hair_mask, threshold=0.15):
    coverage = np.sum(hair_mask > 0) / (hair_mask.shape[0] * hair_mask.shape[1])
    return coverage, coverage > threshold


def preprocess_skin_image(img, resize_shape=(224, 224), hair_threshold=0.15, debug=False):
    hair_removed, hair_mask = remove_hair(img)
    coverage, too_much_hair = check_hair_coverage(hair_mask, hair_threshold)
    
    if too_much_hair:
        print(f"Too much hair detected ({coverage*100:.1f}%). Please retake the photo with less hair obstruction.")
        return None
    
    if debug:
        print(f"Hair coverage: {coverage*100:.2f}%")
        show_image("Hair Removed", hair_removed)
    
    final_img = cv2.resize(hair_removed, resize_shape)
    final_img = cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB)
    final_img = final_img.astype("float32") / 255.0
    
    return final_img



image_path = "/data/raw/HAM10000/metadata/img"
img = cv2.imread(image_path)

processed_img = preprocess_skin_image(img, debug=True)

if processed_img is not None:
    plt.imshow(processed_img)
    plt.title("Final Preprocessed Image for ML")
    plt.axis("off")
    plt.show()